**Final assignment model**

Yunrong Wan

In [1]:
import pandas as pd
import numpy as np


def read_data(fileid): 
    stringvariables=["LOAN_ID","CHANNEL", "FIRST_TIME_HOME_BUYER_IND",
                  "LOAN_PURPOSE", "PROPERTY_TYPE", "OCCUPANCY_STATUS",
                  "PROPERTY_STATE", "ZIP_CODE_SHORT"]
    floatvariables=["NUMBER_OF_UNITS","FORECLOSURE","LTV",
                   "NUMBER_OF_BORROWERS","DTI","LOAN_AGE",
                   "REM_MONTHS_LEGAL_MATURITY", "REM_MONTHS_MATURITY",
                   "ORIGINAL_LOAN_TERM", "CLTV",
                   "B_CREDIT_SCORE_O","CB_CREDIT_SCORE_O", "NMONTHS",
                   "MSA","MORTGAGE_INSURANCE_PERCENTAGE",
                   "ORIGINAL_INTEREST_RATE", "CURRENT_INTEREST_RATE",
                   "ORIGINAL_UPB"]
    datevariables=["MONTHLY_REPORTING_PERIOD",
                   "ORIGINATION_DATE",
                   "FIRST_PAYMENT_DATE",
                   "MATURITY_DATE"]
    objectvariables=stringvariables + datevariables
    dtypes={}
    for s in objectvariables:
        dtypes[s]="object"
    for s in floatvariables:
        dtypes[s]="float"
    na_strings=["".join([" " for j in range(m)]) for m in range(10)]
    df=pd.read_csv(fileid,dtype=dtypes,na_values=na_strings)
    for datevar in datevariables:
        df[datevar]=pd.to_datetime(df[datevar],format="%m%Y")
    return(df)


In [2]:
#loading required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab as pl
#dataset processing methods
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
#loading regression methods
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as qda
from sklearn.naive_bayes import GaussianNB as gnb
from sklearn.tree import DecisionTreeClassifier as dt
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
import xgboost as xgb
#evaluation methods
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,accuracy_score,roc_auc_score,precision_score,recall_score


In [3]:
#split the independent variables and dependent variables
df=read_data("trainTLJC.csv")
y2 = df.FORECLOSURE
nf = sum(y2)
y1 = df.NMONTHS
print("number of foreclosure",nf)
print("NMONTHS mean:",np.mean(y1))
X = df.iloc[:,1:-2]
print(X.dtypes)
#print(X)

number of foreclosure 7708.0
NMONTHS mean: 61.042388
MONTHLY_REPORTING_PERIOD         datetime64[ns]
CHANNEL                                  object
ORIGINAL_INTEREST_RATE                  float64
CURRENT_INTEREST_RATE                   float64
ORIGINAL_UPB                            float64
ORIGINAL_LOAN_TERM                      float64
ORIGINATION_DATE                 datetime64[ns]
FIRST_PAYMENT_DATE               datetime64[ns]
LOAN_AGE                                float64
REM_MONTHS_LEGAL_MATURITY               float64
REM_MONTHS_MATURITY                     float64
MATURITY_DATE                    datetime64[ns]
LTV                                     float64
CLTV                                    float64
NUMBER_OF_BORROWERS                     float64
DTI                                     float64
B_CREDIT_SCORE_O                        float64
CB_CREDIT_SCORE_O                       float64
FIRST_TIME_HOME_BUYER_IND                object
LOAN_PURPOSE                       

In [4]:
df.isnull().sum()

LOAN_ID                               0
MONTHLY_REPORTING_PERIOD              0
CHANNEL                               0
ORIGINAL_INTEREST_RATE                0
CURRENT_INTEREST_RATE                 5
ORIGINAL_UPB                          0
ORIGINAL_LOAN_TERM                    0
ORIGINATION_DATE                      0
FIRST_PAYMENT_DATE                    0
LOAN_AGE                              5
REM_MONTHS_LEGAL_MATURITY             5
REM_MONTHS_MATURITY                   5
MATURITY_DATE                         5
LTV                                   0
CLTV                               3137
NUMBER_OF_BORROWERS                 102
DTI                                8917
B_CREDIT_SCORE_O                   1617
CB_CREDIT_SCORE_O                135292
FIRST_TIME_HOME_BUYER_IND           202
LOAN_PURPOSE                          0
PROPERTY_TYPE                         0
NUMBER_OF_UNITS                       0
OCCUPANCY_STATUS                      0
PROPERTY_STATE                        0


In [5]:
# save all float variables
floatcols=X.dtypes=="float64"
X=X.loc[:,floatcols]
# drop variables which miss too much data
X=X.drop(['CB_CREDIT_SCORE_O','MORTGAGE_INSURANCE_PERCENTAGE'],axis=1)
# save columns name
Xdf=X

In [6]:
# save all data variables
daycols=df.dtypes=="datetime64[ns]"
D=df.loc[:,daycols]

In [7]:
#impute missing data
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
  
Xim=X.values
imp = IterativeImputer(max_iter=1000, random_state=0)
imp=imp.fit(Xim)
Xim=imp.transform(Xim)
X=pd.DataFrame(Xim)
#print(X)

In [8]:
#Test Corr
#print(X.corr())
print(Xdf.corr())
X=X.drop(columns=[1,3,5,7],axis=1)
Xdf=Xdf.drop(columns=Xdf.iloc[:,[1,3,5,7]].columns,axis=1)
#print(X.corr())
print(Xdf.corr())

                           ORIGINAL_INTEREST_RATE  CURRENT_INTEREST_RATE  \
ORIGINAL_INTEREST_RATE                   1.000000               0.999940   
CURRENT_INTEREST_RATE                    0.999940               1.000000   
ORIGINAL_UPB                            -0.115782              -0.115791   
ORIGINAL_LOAN_TERM                       0.375552               0.375598   
LOAN_AGE                                 0.019756               0.019774   
REM_MONTHS_LEGAL_MATURITY                0.375234               0.375279   
REM_MONTHS_MATURITY                      0.373475               0.373520   
LTV                                      0.252844               0.252854   
CLTV                                     0.232992               0.233003   
NUMBER_OF_BORROWERS                     -0.061727              -0.061701   
DTI                                      0.084704               0.084748   
B_CREDIT_SCORE_O                        -0.162449              -0.162386   
NUMBER_OF_UN

In [9]:
#preprocess object to dummies
channel_dummies=pd.get_dummies(df["CHANNEL"],drop_first=True,prefix="channel")
firsttime_dummies=pd.get_dummies(df["FIRST_TIME_HOME_BUYER_IND"],drop_first=True,prefix="firsttime")                
loanpurpose_dummies=pd.get_dummies(df["LOAN_PURPOSE"],drop_first=True,prefix="loanpurpose") 
propertytype_dummies=pd.get_dummies(df["PROPERTY_TYPE"],drop_first=True,prefix="propertytype")
occupancy_dummies=pd.get_dummies(df["OCCUPANCY_STATUS"],drop_first=True,prefix="occupancy")          
propertystate_dummies=pd.get_dummies(df["PROPERTY_STATE"],drop_first=True,prefix="propertystate") 

In [10]:
#combine the X with dummies variables
Xall= pd.concat([Xdf,channel_dummies,firsttime_dummies,loanpurpose_dummies,propertytype_dummies,occupancy_dummies,propertystate_dummies],axis=1)

In [11]:
#split the train set and test set
X_train, X_test,y1_train,y1_test,y2_train,y2_test=train_test_split(X,y1,y2,test_size=0.2,random_state=21)
X_train=X_train.values
X_test=X_test.values
channel_train,channel_test=train_test_split(channel_dummies,test_size=0.2,random_state=21)
loanpurpose_train,loanpurpose_test=train_test_split(loanpurpose_dummies,test_size=0.2,random_state=21)
firsttime_train,firsttime_test=train_test_split(firsttime_dummies,test_size=0.2,random_state=21)
propertytype_train,propertytype_test=train_test_split(propertytype_dummies,test_size=0.2,random_state=21)
occupancy_train,occupancy_test=train_test_split(occupancy_dummies,test_size=0.2,random_state=21)
propertystate_train,propertystate_test=train_test_split(propertystate_dummies,test_size=0.2,random_state=21)

**Predicting model of NMONTHS**

In [12]:
#select regression model roughly
model=[LinearRegression(),Ridge(),lda(),gnb(),dt(),Lasso(),ElasticNet()]
for i in model:
    clf=i.fit(X_train,y1_train)
    a=mean_absolute_error(y1_test, clf.predict(X_test))
    print(a,i)

37.91916646379718 LinearRegression()
37.91917908556337 Ridge()
50.1854 LinearDiscriminantAnalysis()
62.51686 GaussianNB()
49.20718 DecisionTreeClassifier()
38.12247041671439 Lasso()
39.516300821875966 ElasticNet()


In [13]:
##### Select parameter for NMONTHS ####
#Add the float variables one by one to calculate the error
error=[]
parameter1=[]
acu=np.mean(y1)
X_sitrain=np.zeros(X_train.shape[0]).reshape(-1,1)
X_sitest=np.zeros(X_test.shape[0]).reshape(-1,1)
Xte=Xdf
while (X_train.shape[1]>0):
    for i in range(X_train.shape[1]):
        X_strain=np.append(X_sitrain,X_train[:,i].reshape(-1,1),axis=1)
        X_stest=np.append(X_sitest,X_test[:,i].reshape(-1,1),axis=1)
        #print(X_sitrain)
        #print(X_strain)
        linear=LinearRegression().fit(X_strain,y1_train)
        #ridge=Ridge().fit(X_sitrain,y1_train)
        a=mean_absolute_error(y1_test, linear.predict(X_stest))
        #a=r2_score(y1_test, linear.predict(X_sitest))
        #print(a,list(Xte.iloc[:,i:i+1]))
        error.append([a,list(Xte.iloc[:,i:i+1].columns),i,X_strain,X_stest])
    error.sort(reverse=False)
    if error[0][0]< acu:
        parameter1.append(error[0][1][0])
        acu=error[0][0]
        Xte=Xte.drop(error[0][1][0],axis=1)
        X_train=np.delete(X_train,error[0][2],axis=1)
        X_test=np.delete(X_test,error[0][2],axis=1)
        X_sitrain=error[0][3]
        X_sitest=error[0][4]
        #print(parameter1)
        error=[]
    else:
        break
float_parameter1=parameter1[:]#save float parameter
print("Done!")
print("selected float parameter:",parameter1)
print("MAE",acu)

Done!
selected float parameter: ['ORIGINAL_INTEREST_RATE', 'ORIGINAL_UPB', 'REM_MONTHS_MATURITY', 'B_CREDIT_SCORE_O', 'NUMBER_OF_UNITS', 'NUMBER_OF_BORROWERS']
MAE 37.91201958817663


In [14]:
# Add the dummy variables one by one to calculate the error,based on parameter above
dummy_train=[channel_train,loanpurpose_train,firsttime_train,propertytype_train,occupancy_train,propertystate_train]
dummy_test=[channel_test,loanpurpose_test,firsttime_test,propertytype_test,occupancy_test,propertystate_test]
dummy_parameter1=[]
error=[]
while (len(dummy_train)>0):
    for i in range(len(dummy_train)):
        X_strain=np.append(X_sitrain,dummy_train[i],axis=1)
        X_stest=np.append(X_sitest,dummy_test[i],axis=1)
        #print(X_strain)
        linear=LinearRegression().fit(X_strain,y1_train)
        #ridge=Ridge().fit(X_sitrain,y1_train)
        a=mean_absolute_error(y1_test, linear.predict(X_stest))
        #a=r2_score(y1_test, linear.predict(X_sitest))
        #print(a,list(dummy_train[i].columns))
        error.append([a,list(dummy_train[i].columns),i,X_strain,X_stest])
    error.sort(reverse=False)
    if error[0][0]< acu:
        p=error[0][1]
        for t in range(len(p)):
            parameter1.append(p[t])
            dummy_parameter1.append(p[t])#save dummy parameters
        acu=error[0][0]
        del dummy_train[error[0][2]]
        del dummy_test[error[0][2]]
        X_sitrain=error[0][3]
        X_sitest=error[0][4]
        #print(dummy_parameter1)
        error=[]
    else: 
        break
print("Done!")
print("selected float&dummy parameter:",parameter1)
print("MAE",acu)

Done!
selected float&dummy parameter: ['ORIGINAL_INTEREST_RATE', 'ORIGINAL_UPB', 'REM_MONTHS_MATURITY', 'B_CREDIT_SCORE_O', 'NUMBER_OF_UNITS', 'NUMBER_OF_BORROWERS', 'propertystate_AL', 'propertystate_AR', 'propertystate_AZ', 'propertystate_CA', 'propertystate_CO', 'propertystate_CT', 'propertystate_DC', 'propertystate_DE', 'propertystate_FL', 'propertystate_GA', 'propertystate_GU', 'propertystate_HI', 'propertystate_IA', 'propertystate_ID', 'propertystate_IL', 'propertystate_IN', 'propertystate_KS', 'propertystate_KY', 'propertystate_LA', 'propertystate_MA', 'propertystate_MD', 'propertystate_ME', 'propertystate_MI', 'propertystate_MN', 'propertystate_MO', 'propertystate_MS', 'propertystate_MT', 'propertystate_NC', 'propertystate_ND', 'propertystate_NE', 'propertystate_NH', 'propertystate_NJ', 'propertystate_NM', 'propertystate_NV', 'propertystate_NY', 'propertystate_OH', 'propertystate_OK', 'propertystate_OR', 'propertystate_PA', 'propertystate_PR', 'propertystate_RI', 'propertystate

In [15]:
#build dataset of day_gap
day_gap=pd.DataFrame()
dayvariable=[]
n=0
for i in range(4):
    j=4-i
    for t in range(1,j):
        n+=1
        Xday=abs(D.iloc[:,i]-D.iloc[:,i+t])
        #print(Xday)
        Xday=Xday.astype('timedelta64[M]').astype(float)
        day_gap=pd.concat([day_gap,Xday],axis=1)
        dayvariable.append([list(D.iloc[:,i:i+1].columns),list(D.iloc[:,i+t:i+t+1].columns),n-1])
day_gap.columns=['R-O','R-F','R-M','O-F','O-M','F-M']

In [16]:
print(day_gap.corr())
day_gap=day_gap.loc[:,['R-M','O-F']]
print(day_gap.corr())

          R-O       R-F       R-M       O-F       O-M       F-M
R-O  1.000000  0.618616 -0.002829  0.128937  0.016119  0.015496
R-F  0.618616  1.000000 -0.048651  0.051186 -0.036030 -0.036353
R-M -0.002829 -0.048651  1.000000 -0.008031  0.999804  0.999809
O-F  0.128937  0.051186 -0.008031  1.000000 -0.006027 -0.009131
O-M  0.016119 -0.036030  0.999804 -0.006027  1.000000  0.999985
F-M  0.015496 -0.036353  0.999809 -0.009131  0.999985  1.000000
          R-M       O-F
R-M  1.000000 -0.008031
O-F -0.008031  1.000000


In [17]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
dim=day_gap.values
imp = IterativeImputer(max_iter=1000, random_state=0)
imp=imp.fit(dim)
dim=imp.transform(dim)
day_gap=pd.DataFrame(dim)
day_gap.columns=['R-M','O-F']

In [18]:
x=Xdf.loc[:,float_parameter1]
#print(x)
#print(x.corr())
#combine the selected X with dummies variables
Xim_sel=x.values
imp = IterativeImputer(max_iter=1000, random_state=0)
imp=imp.fit(Xim_sel)
Xim_sel=imp.transform(Xim_sel)
X_sel=pd.DataFrame(Xim_sel)
print("corr:","\n",X_sel.corr())
X_sel = pd.concat([X_sel,Xall.loc[:,dummy_parameter1],day_gap],axis=1)
#print(X_sel)


corr: 
           0         1         2         3         4         5
0  1.000000 -0.115782  0.373475 -0.163343  0.036092 -0.061740
1 -0.115782  1.000000  0.219349  0.041549  0.061036  0.139893
2  0.373475  0.219349  1.000000 -0.099367  0.001187 -0.078425
3 -0.163343  0.041549 -0.099367  1.000000 -0.006006  0.076406
4  0.036092  0.061036  0.001187 -0.006006  1.000000 -0.018386
5 -0.061740  0.139893 -0.078425  0.076406 -0.018386  1.000000


In [19]:
X_sel_train,X_sel_test=train_test_split(X_sel,test_size=0.2,random_state=21)
X_sel_train=X_sel_train.values
X_sel_test=X_sel_test.values

**Evaluation of testing set**

In [20]:
linear=LinearRegression().fit(X_sel_train,y1_train)
Error_a=mean_absolute_error(y1_test, linear.predict(X_sel_test))
Error_s=mean_squared_error(y1_test, linear.predict(X_sel_test))
Error_r=r2_score(y1_test, linear.predict(X_sel_test))
print("mean absolute error:",Error_a)
print("mean squared error:",Error_s)
print("r2 scorce:",Error_r)

mean absolute error: 37.35503466978695
mean squared error: 2325.6128026579554
r2 scorce: 0.1543288574411008


**Forecast Training set**

In [21]:
dtest=read_data("test_predsTLJC.csv")
G=dtest.loc[:,float_parameter1]
Gdf=G
Gim=G.values
imp = IterativeImputer(max_iter=1000, random_state=5)
imp=imp.fit(Gim)
Gim=imp.transform(Gim)
G=pd.DataFrame(Gim)
#date data
daycols=dtest.dtypes=="datetime64[ns]"
GD=dtest.loc[:,daycols]
Xday1=abs(GD.iloc[:,0]-GD.iloc[:,3])
Xday1=Xday1.astype('timedelta64[M]').astype(float)
Xday2=abs(GD.iloc[:,1]-GD.iloc[:,2])
Xday2=Xday2.astype('timedelta64[M]').astype(float)
G_day_gap=pd.concat([Xday1,Xday2],axis=1)
Gdim=G_day_gap.values
imp = IterativeImputer(max_iter=1000, random_state=0)
imp=imp.fit(Gdim)
Gdim=imp.transform(Gdim)
G_day_gap=pd.DataFrame(Gdim)
G_day_gap.columns=['R-M','O-F']
#preprocess object to dummies
G_channel_dummies=pd.get_dummies(dtest["CHANNEL"],drop_first=True,prefix="channel")
G_firsttime_dummies=pd.get_dummies(dtest["FIRST_TIME_HOME_BUYER_IND"],drop_first=True,prefix="firsttime")                
G_loanpurpose_dummies=pd.get_dummies(dtest["LOAN_PURPOSE"],drop_first=True,prefix="loanpurpose") 
G_propertytype_dummies=pd.get_dummies(dtest["PROPERTY_TYPE"],drop_first=True,prefix="propertytype")
G_occupancy_dummies=pd.get_dummies(dtest["OCCUPANCY_STATUS"],drop_first=True,prefix="occupancy")          
G_propertystate_dummies=pd.get_dummies(dtest["PROPERTY_STATE"],drop_first=True,prefix="propertystate") 
Gall= pd.concat([Gdf,G_channel_dummies,G_firsttime_dummies,G_loanpurpose_dummies,G_propertytype_dummies,G_occupancy_dummies,G_propertystate_dummies],axis=1)

G_test = pd.concat([G,Gall.loc[:,dummy_parameter1],G_day_gap],axis=1).values


In [22]:
Gy1=linear.predict(G_test)
Gy1=pd.DataFrame(Gy1.reshape(-1,1))
Gy1.columns=['NMONTHS']
ID=dtest.iloc[:,0]
test=pd.concat([ID,Gy1],axis=1)
print(test)

            LOAN_ID    NMONTHS
0      ICIHCHIADIIC  32.668253
1      HBBAFDJBJJEC  77.404789
2      CGEBEHJDCCBG  51.100108
3      HCDGFDIDAEGE  97.703619
4      AGAHJDCEEFEI  48.699685
...             ...        ...
99995  CDEHJHAGEGJC  76.794369
99996  IAAHCBBFDCCJ  84.431207
99997  JECEDABADAHF  57.180248
99998  FCDJIGJCGAJF  39.668084
99999  CDEHHGFEICAF  81.185956

[100000 rows x 2 columns]


**Predicting model of FORECLOSURE**

In [23]:
# Predicting model for foreclosure
# split the train set and test set
# standard data
ss=StandardScaler()
X_ss=ss.fit_transform(X)
X_train, X_test,y1_train,y1_test,y2_train,y2_test=train_test_split(X_ss,y1,y2,test_size=0.2,random_state=21)
X_train=X_train
X_test=X_test


channel_train,channel_test=train_test_split(channel_dummies,test_size=0.2,random_state=21)
loanpurpose_train,loanpurpose_test=train_test_split(loanpurpose_dummies,test_size=0.2,random_state=21)
firsttime_train,firsttime_test=train_test_split(firsttime_dummies,test_size=0.2,random_state=21)
propertytype_train,propertytype_test=train_test_split(propertytype_dummies,test_size=0.2,random_state=21)
occupancy_train,occupancy_test=train_test_split(occupancy_dummies,test_size=0.2,random_state=21)
propertystate_train,propertystate_test=train_test_split(propertystate_dummies,test_size=0.2,random_state=21)



In [24]:
#gridsearch for best parameters
from sklearn.model_selection import GridSearchCV
parameters = {'solver':('sag', 'lbfgs','newton-cg'), 'C':[0.5, 0.1]}
logistic = LogisticRegression()
clf = GridSearchCV(logistic, parameters)
clf.fit(X_train, y2_train)
sorted(clf.cv_results_.keys())
print(clf.best_params_)

{'C': 0.5, 'solver': 'sag'}


In [25]:
# auc of all float variables
logistic=LogisticRegression(solver='sag',multi_class='multinomial',class_weight='balanced',random_state=2,C=.5)
logistic.fit(X_train,y2_train)
y2pred_prob=logistic.predict_proba(X_test)
y2_pred=logistic.predict(X_test)
pd.crosstab(y2_test,y2_pred)
#print(roc_auc_score(y2_test,y2_pred))

#delete variables
error=[]
Xte=Xdf
X_sitrain=X_train
X_sitest=X_test
acu=precision_score(y2_test,y2_pred)
parameter2=[]
while (X_sitrain.shape[1]>0):
    for i in range(X_sitrain.shape[1]):
        X_strain=np.delete(X_sitrain,i,axis=1)
        X_stest=np.delete(X_sitest,i,axis=1)
        logistic=LogisticRegression(solver='sag',multi_class='multinomial',class_weight='balanced',random_state=2,C=.5)
        logistic.fit(X_sitrain,y2_train)
        y2pred_prob=logistic.predict_proba(X_sitest)
        y2_pred=logistic.predict(X_sitest)
        a=precision_score(y2_test,y2_pred)
        #print(a,list(Xte.iloc[:,i:i+1]))
        error.append([a,list(Xte.iloc[:,i:i+1].columns),i,X_strain,X_stest])
    error.sort(reverse=True)
    if error[0][0]> acu:
        parameter2.append(error[0][1][0])
        acu=error[0][0]
        Xte=Xte.drop(error[0][1][0],axis=1)
        X_sitrain=error[0][3]
        X_sitest=error[0][4]
        #print(parameter2)
        error=[]
    else:
        
        break
print("Done!")
print("deleted float parameter:",parameter2)
print("precision score",acu)


#select dummies variables
# Add the dummy variables one by one to calculate the error,based on parameter above
dummy_train=[channel_train,loanpurpose_train,firsttime_train,propertytype_train,occupancy_train,propertystate_train]
dummy_test=[channel_test,loanpurpose_test,firsttime_test,propertytype_test,occupancy_test,propertystate_test]
dummy_parameter2=[]
error=[]
X_sitrain=X_train
X_sitest=X_test
while (len(dummy_train)>0):
    for i in range(len(dummy_train)):
        X_strain=np.append(X_sitrain,dummy_train[i],axis=1)
        X_stest=np.append(X_sitest,dummy_test[i],axis=1)
        #print(X_strain)
        logistic=LogisticRegression(solver='sag',multi_class='multinomial',class_weight='balanced',random_state=2,C=.5)
        logistic.fit(X_strain,y2_train)
        y2pred_prob=logistic.predict_proba(X_stest)
        y2_pred=logistic.predict(X_stest)
        a=precision_score(y2_test,y2_pred)
        #print(a,list(dummy_train[i].columns))
        error.append([a,list(dummy_train[i].columns),i,X_strain,X_stest])
        #print(error)
    error.sort(reverse=True)
    if error[0][0]> acu:
        p=error[0][1]
        for t in range(len(p)):
            parameter2.append(p[t])
            dummy_parameter2.append(p[t])#save dummy parameters
        acu=error[0][0]
        del dummy_train[error[0][2]]
        del dummy_test[error[0][2]]
        X_sitrain=error[0][3]
        X_sitest=error[0][4]
        #print(dummyparameter2)
        error=[]
    else: 
        break
print("Done!")
print("selected dummy parameter:",parameter2)
print("precision score",acu)


Done!
deleted float parameter: []
precision score 0.06748785565579458


/Users/rongnozomi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/rongnozomi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/rongnozomi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/rongnozomi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/rongnozomi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/rongnozomi/opt/ana

Done!
selected dummy parameter: ['propertystate_AL', 'propertystate_AR', 'propertystate_AZ', 'propertystate_CA', 'propertystate_CO', 'propertystate_CT', 'propertystate_DC', 'propertystate_DE', 'propertystate_FL', 'propertystate_GA', 'propertystate_GU', 'propertystate_HI', 'propertystate_IA', 'propertystate_ID', 'propertystate_IL', 'propertystate_IN', 'propertystate_KS', 'propertystate_KY', 'propertystate_LA', 'propertystate_MA', 'propertystate_MD', 'propertystate_ME', 'propertystate_MI', 'propertystate_MN', 'propertystate_MO', 'propertystate_MS', 'propertystate_MT', 'propertystate_NC', 'propertystate_ND', 'propertystate_NE', 'propertystate_NH', 'propertystate_NJ', 'propertystate_NM', 'propertystate_NV', 'propertystate_NY', 'propertystate_OH', 'propertystate_OK', 'propertystate_OR', 'propertystate_PA', 'propertystate_PR', 'propertystate_RI', 'propertystate_SC', 'propertystate_SD', 'propertystate_TN', 'propertystate_TX', 'propertystate_UT', 'propertystate_VA', 'propertystate_VI', 'proper

/Users/rongnozomi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


**Evaluation of training set**

In [28]:
#final selected dataset
#standard data
X_ss=pd.DataFrame(X_ss)
X_sel = pd.concat([X_ss,Xall.loc[:,parameter2]],axis=1)
#split the dataset as train set and test set=100,000
X_sel_train,X_sel_test=train_test_split(X_sel,test_size=0.4,random_state=20)
X_train, X_test,y1_train,y1_test,y2_train,y2_test=train_test_split(X,y1,y2,test_size=0.4,random_state=20)
X_sel_train=X_sel_train.values
X_sel_test=X_sel_test.values
#logistical regression
logistic=LogisticRegression(solver='sag',multi_class='multinomial',class_weight='balanced',random_state=2,C=.5)
logistic.fit(X_sel_train,y2_train)
y2pred_prob=logistic.predict_proba(X_sel_test)
y2_pred=logistic.predict(X_sel_test)
print(precision_score(y2_test,y2_pred))
pd.crosstab(y2_test,y2_pred)

0.07859365688935974


/Users/rongnozomi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


col_0,0.0,1.0
FORECLOSURE,,
0.0,69108,27832
1.0,686,2374


In [29]:
#select 1000 foreclosure
sort=np.sort(y2pred_prob[:,1])
l=len(X_sel_test)
n=0
threshold=sort[-1000]
gy2=np.zeros(l)
for i in range(l):
    if y2pred_prob[:,1][i]<threshold:
        gy2[i]=0
    else:
        gy2[i]=1 
        n+=1
print("roc_auc_score",roc_auc_score(y2_test,gy2))
print("precision_score",precision_score(y2_test,gy2))
print("recall_score",recall_score(y2_test,gy2))
pd.crosstab(y2_test,gy2)

roc_auc_score 0.5295648140282178
precision_score 0.206
recall_score 0.0673202614379085


col_0,0.0,1.0
FORECLOSURE,,
0.0,96146,794
1.0,2854,206


**Forecast the testing set**

In [30]:
g=dtest.loc[:,Xdf.columns]
gim=g.values
imp = IterativeImputer(max_iter=1000, random_state=5)
imp=imp.fit(gim)
gim=imp.transform(gim)
ss=StandardScaler()
g_ss=ss.fit_transform(gim)
g=pd.DataFrame(g_ss)
g_test = pd.concat([g,Gall.loc[:,dummy_parameter2]],axis=1).values
print(g_test)

[[ 1.13387179e+00 -6.46027549e-04 -3.46509120e-01 ...  1.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [-8.48739768e-01  1.67145338e+00 -3.46509120e-01 ...  1.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [ 1.43888895e+00 -9.32244267e-01 -3.46509120e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [-8.61968619e-02 -1.13528491e+00 -9.66458436e-01 ...  1.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [ 5.23837463e-01  2.38225316e-01  2.73440196e-01 ...  1.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.00124835e+00 -8.96413566e-01 -3.46509120e-01 ...  1.00000000e+00
   0.00000000e+00  1.00000000e+00]]


In [31]:
Gy2pred_prob=logistic.predict_proba(g_test)
print(Gy2pred_prob)

[[0.93679632 0.06320368]
 [0.54383767 0.45616233]
 [0.32442479 0.67557521]
 ...
 [0.99297778 0.00702222]
 [0.9025011  0.0974989 ]
 [0.75413855 0.24586145]]


In [32]:
ysort=np.sort(Gy2pred_prob[:,1])
l=len(g_test)
n=0
threshold=ysort[-1000]
Gy2=np.zeros(l)
for i in range(l):
    if Gy2pred_prob[:,1][i]<threshold:
        Gy2[i]=0
    else:
        Gy2[i]=1 
        n+=1

Gy2=pd.DataFrame(Gy2.reshape(-1,1))
Gy2.columns=['FORECLOSURE']
test2=pd.concat([test,Gy2],axis=1)
print(test2)

            LOAN_ID    NMONTHS  FORECLOSURE
0      ICIHCHIADIIC  32.668253          0.0
1      HBBAFDJBJJEC  77.404789          0.0
2      CGEBEHJDCCBG  51.100108          0.0
3      HCDGFDIDAEGE  97.703619          0.0
4      AGAHJDCEEFEI  48.699685          0.0
...             ...        ...          ...
99995  CDEHJHAGEGJC  76.794369          0.0
99996  IAAHCBBFDCCJ  84.431207          0.0
99997  JECEDABADAHF  57.180248          0.0
99998  FCDJIGJCGAJF  39.668084          0.0
99999  CDEHHGFEICAF  81.185956          0.0

[100000 rows x 3 columns]


In [33]:
test2.to_csv("preds.csv",index=False)

**Test the vaildation of preds file**

In [36]:
import pandas as pd
import re

def test_prediction_file(fileid):
    try:
        dtypes={"LOAN_ID":"object","NMONTHS":"float","FORECLOSURE":"float"}
        df=pd.read_csv(fileid,dtype=dtypes)
    except:
        print("Error: Your file is unreadable")
        return

    def LIDcheck(x):
        if re.match("[A-Z]{12}",x):
            return(True)
        return(False)
    
    
    if df.shape[0]!=100000:
        print("Error: your file should have 100,000 rows")
        print("       not including the header row")
        return
    if df.shape[1]<3:
        print("Error: your file does not have enough columns")
        return
    if "LOAN_ID" not in df.columns:
        print("Error: your file does not have a LOAN_ID column")
    else:
        LIDCHECKS=df.LOAN_ID.apply(LIDcheck).sum()==100000
        if not LIDCHECKS:
            print("Error: Your LOAN_ID's should all be")
            print("       consist of upper case alphabetic characters")
            return
        if df.dtypes.LOAN_ID!="object":
            print("Error: your LOAN_ID column is not the correct type")
            print("       should be 'object'")
            return
        if df.LOAN_ID.nunique()!=100000:
            print("Error: You do not have unique LOAN_ID values")
            return
        if df.LOAN_ID.isnull().sum()>0:
            print("Error: You have missing LOAN_ID's")
        if df.LOAN_ID.apply(len).apply(lambda x:x==12).sum()!=100000:
            print("Error: All LOAN_ID's should have length 12")
            return
        
    if "NMONTHS" not in df.columns:
        print("Error: your file does not have an NMONTHS column")
        return
    if df.NMONTHS.isnull().sum()>0:
        print("Error: you have missing values in your NMONTHS column")
        return
    if "FORECLOSURE" not in df.columns:
        print("Error: your file does not have a FORECLOSURE column")
        return
    if df.FORECLOSURE.isnull().sum()>0:
        print("Error: you have missing values in your FORECLOSURE column")
    if (df.FORECLOSURE==0.).sum()!=99000:
        print("Error: your FORECLOSURE column should contain 99,000 zeros")
    if (df.FORECLOSURE==1.).sum()!=1000:
        print("Error: your FORECLOSURE column should contain 1,000 ones")

In [37]:
test_prediction_file("preds.csv")